<a href="https://colab.research.google.com/github/Shona173/codes/blob/main/Comparing_Bounding_brending_and_linear_brending3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install POT
!pip install ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import numpy.random as random

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from google.colab import files
from matplotlib import animation
from IPython.display import HTML
import ot

In [3]:
def clamp(value, min_val, max_val):
    return np.maximum(min_val, np.minimum(value, max_val))

In [15]:
def sdf_box(p, b=[0.7,0.7]):
    p = p.copy()
    d = np.abs(x)-b
    max_d = np.maximum(d, 0.0)
    length_d = np.sqrt(np.sum(max_d**2, 1))
    return length_d + np.minimum(np.maximum(d[:,0], d[:,1]), 0.0)

In [20]:
def sdf_blade(p):
    p = p.copy()
    blade_size = np.array([0.05, 0.3])
    return sdf_box(p - np.array([0.0, 0.15]), blade_size)

def sdf_guard(p):
    p = p.copy()
    guard_size = np.array([0.1, 0.02])
    return sdf_box(p - np.array([0.0, 0.05]), guard_size)

def sdf_handle(p):
    p = p.copy()
    handle_size = np.array([0.02, 0.1])
    return sdf_box(p - np.array([0.0, -0.05]), handle_size)

def sdf_sword(p):
    p = p.copy()
    return np.minimum(np.minimum(sdf_blade(p), sdf_guard(p)), sdf_handle(p))

In [5]:
def sdf_heart(p,r=0.5):
    p=p.copy()
    p[0] = abs(p[0])
    if p[1] + p[0] > 1.0:
        return np.sqrt(np.dot(p - np.array([0.25, 0.75]))) - np.sqrt(2.0) / 4.0
    return np.sqrt(min(np.dot(p - np.array([0.00, 1.00])),np.dot(p - 0.5 * max(p[:0] + p[:1], 0.0)))) * np.sign(p[:0]-p[:1])

In [6]:
def gen_grid(resolution, low=-1.0, high=1.0):
    idx = np.linspace(low, high, num=resolution)
    x, y = np.meshgrid(idx, idx)
    V = np.concatenate((x.reshape((-1,1)), y.reshape((-1,1))), 1)

    return V

In [7]:
def r_union(f1, f2):
    return f1 + f2 + np.sqrt(f1**2 + f2**2)

In [8]:
def r_intersection(f1, f2):
    return f1 + f2 - np.sqrt(f1**2 + f2**2)

In [9]:
def linear_morphing(f1, f2, t, x):
    return (1-t)*f1(x) + t*f2(x)

In [10]:
def space_time_blending(f1, f2, t, x, a0=1.0, a1=1.0, a2=1.0):
    f1x = f1(x)
    f2x = f2(x)

    #f1x_cyl = np.minimum(f1x, -t) # needs R-functions to work properly
    f1x_cyl = r_intersection(f1x, -t)
    #f2x_cyl = np.minimum(f2x, (t-1)) # needs R-functions to work properly
    f2x_cyl = r_intersection(f2x, (t-1))

    disp = a0 / (1.0 + (f1x_cyl/a1)**2 + (f2x_cyl/a2)**2)
    r_uni = r_union(f1x_cyl, f2x_cyl)

    return r_uni + disp

In [11]:
def bounded_blending(f1, f2, t, x, a0=1.0, a1=1.0, a2=1.0):
  f1x=f1(x)
  f2x=f2(x)
  tx=np.clip(t(x), 0, 1)
  blend=a0*(1-tx)**2+a1*(1-tx)*tx+a2*tx**2
  return (1-blend)*f1x+blend*f2x

In [12]:
def sample_and_normalize(f, grid, grid_size):
    '''
    Sample f on the grid and normalize it.
    Assume f>0 outside and <0 inside.
    '''
    print(grid.shape)
    fv = f(grid)
    print(fv.shape)

    # >0 inside
    fv = -fv

    # f is the characteristic function for {f>0}
    fv[fv>=0.0] = 1.0
    fv[fv<0.0] = 0.0

    total_sum = np.sum(fv)
    if total_sum > 0:
        fv = fv / total_sum
    else:
        raise ValueError("The sum of the function values is zero; normalization is not possible.")

    # reshape to have the same shape as grid
    fv = fv.reshape(grid_size, grid_size)

    return fv

In [21]:
grid_size = 64
grid = gen_grid(grid_size,-2.0,2.0)

# f1 and f2 are prob. distribution corresponding to f1 and f2
f1 = sample_and_normalize(sdf_sword, grid, grid_size)
f2 = sample_and_normalize(sdf_heart, grid, grid_size)

A = np.array([f1,f2])

nb_images = 5
reg = 0.004

v1 = np.array((1, 0))
v2 = np.array((0, 1))

fig, axes = plt.subplots(1, nb_images, figsize=(7, 7))
plt.suptitle("Optimal Trasport")
cm = "Blues"

for i in range(nb_images):
    tx = float(i) / (nb_images - 1)

    weights = (1 - tx) * v1 + tx * v2

    if i == 0:
        axes[i].imshow(f1, cmap=cm)
    elif i == (nb_images - 1):
        axes[i].imshow(f2, cmap=cm)
    else:
        # call to barycenter computation
        axes[i].imshow(
            ot.bregman.convolutional_barycenter2d(A, reg, weights), cmap=cm
        )
    axes[i].axis("off")
#Optimal Trasport
plt.tight_layout()
plt.show()

(4096, 2)


NameError: name 'x' is not defined